In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
with open("C:/Users/saisn/OneDrive/Desktop/parkinsons.csv") as file:
    data=file.read()

In [14]:
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer

#     non_html=re.sub('<.*?>',' ',x)   
#remove html tag using regular expresion
soup=BeautifulSoup(data,'html.parser')
#remove html tag using beautifulsoup
non_html_text=soup.get_text()
    
#remove unwanted charectors and symbols
text=re.sub('[^a-zA-Z0-9\s]',' ',non_html_text)
text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')

#remove extra spaces
z = []
for i in text.split():
    if i not in z:
        z.append(i)  
text = ' '.join(z)


#tokenize text
    
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
sequence=tokenizer.texts_to_sequences([text])[0]

sequence[:15]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [15]:
ip_dim=len(tokenizer.word_index)+1
ip_dim

3520

In [16]:
#traning sequence

sequences = []

for i in range(3, len(sequence)):
    words = sequence[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)


x,y=[],[]
for i in sequences:
    x.append(i[0:3])
    y.append(i[3])
x=np.array(x)
y=np.array(y)

The Length of sequences are:  3516


In [17]:
x[:10],y[:10]

(array([[ 1,  2,  3],
        [ 2,  3,  4],
        [ 3,  4,  5],
        [ 4,  5,  6],
        [ 5,  6,  7],
        [ 6,  7,  8],
        [ 7,  8,  9],
        [ 8,  9, 10],
        [ 9, 10, 11],
        [10, 11, 12]]),
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]))

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# # # Pad "sequences" to a fixed length
# max_len = 10
# x = pad_sequences(x, maxlen=max_len)
# y = pad_sequences(y, maxlen=max_len)

# Convert y to categorical

y = to_categorical(y, num_classes=ip_dim)
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
model=Sequential()
model.add(Embedding(ip_dim,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(ip_dim,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             35200     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 3520)              3523520   
                                                                 
Total params: 16,607,720
Trainable params: 16,607,720
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
model.fit(x,y,batch_size=62,epochs=70)

Epoch 1/70
57/57 [==============================] - 10s 108ms/step - loss: 8.1776 - accuracy: 0.0000e+00
Epoch 2/70
57/57 [==============================] - 6s 107ms/step - loss: 8.1691 - accuracy: 0.0000e+00
Epoch 3/70
57/57 [==============================] - 6s 111ms/step - loss: 8.1689 - accuracy: 0.0000e+00
Epoch 4/70
57/57 [==============================] - 6s 105ms/step - loss: 8.1688 - accuracy: 0.0000e+00
Epoch 5/70
57/57 [==============================] - 6s 102ms/step - loss: 8.1688 - accuracy: 0.0000e+00
Epoch 6/70
57/57 [==============================] - 6s 102ms/step - loss: 8.1687 - accuracy: 0.0000e+00
Epoch 7/70
57/57 [==============================] - 6s 103ms/step - loss: 8.1687 - accuracy: 0.0000e+00
Epoch 8/70
57/57 [==============================] - 6s 102ms/step - loss: 8.1686 - accuracy: 0.0000e+00
Epoch 9/70
57/57 [==============================] - 6s 103ms/step - loss: 8.1686 - accuracy: 0.0000e+00
Epoch 10/70
57/57 [==============================] - 6s 102ms/s

In [22]:
# For preprocessing input text for test
def preprocess(text):
    soup1=BeautifulSoup(text,'html.parser')#remove html tag using beautifulsoup
    non_html_txt=soup.get_text()
    
    #remove unwanted charectors and symbols
    text=re.sub('[^a-zA-Z0-9\s]',' ',non_html_txt)
    text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')

    #remove extra spaces
    z = []
    for i in text.split():
        if i not in z:
            z.append(i)  
    text = ' '.join(z)
    return text

In [23]:
# It gives single predicted words
def predict_nxt_word(txt):
#      
    
    tok_text = tokenizer.texts_to_sequences([ txt])
#     print(tok_text)
    preds = model.predict(np.array(tok_text), verbose=0)[0]
    next_word = tokenizer.sequences_to_texts([[np.argmax(preds)]])[0]
    return next_word

In [24]:
# It gives 5 predicted words

def predict_nxt_words1(txt, k=5):
    tok_text = tokenizer.texts_to_sequences([txt])
    preds = model.predict(np.array(tok_text), verbose=0)[0]
    top_k_indices = preds.argsort()[-k:][::-1]
    top_k_words = [tokenizer.sequences_to_texts([[i]])[0] for i in top_k_indices]
    return top_k_words

In [25]:
# #Rue this code


# while(True):
#     text=input('Enter Text or 0 for exit=')
#     if text=='0':
#         print('Exited')
#         break
#     else:
#         try:
#             text=preprocess(text)
#             text=text.split()
#             print(predict_nxt_words1(text[-3:]))
#         except Exception as e:
#             print('Error',e)
#             continue

In [26]:
txt='i took me'
txt=preprocess(txt)
txt=txt.split(' ')
txt=txt[-3:]
# print(txt)
nxt=predict_nxt_words1(txt)
print(nxt)

['178', '29000', '67100', '216', '440040']
